# 1. Simple Integer Arithmetic Problem


### 1.1 Problem Statement
Write a program that can parse a simple integer arithmetic expression to give an integer result

### 1.2 Example
"1 + 2 * 3" should yield the answer 7.

### 1.3 Setup
You must be able to run the program such that you can provide the name of a file with one expression per line and for each line output "<expr> = <answer>".
    
### 1.4 Conditions
The expression should support operators +, -, *, /, integers, parentheses.

Implement and test your solution to illustrate how you would implement production code.

You may use 3rd party libraries to implement parts of your solution (e.g. a parser library) but you should not use a 3rd party library that provides the entire solution.

You may implement this in any language, ideally .NET, with F#, but any is fine
### 1.5 Submission
Please submit via GitHub or a similar repository from which the solution can be downloaded.

# 2. The Shunting Yard Algorithm

https://www.cs.utexas.edu/~EWD/MCReps/MR35.PDF

Edsger Dijkstra developed his "Shunting Yard" algorithm to convert an infix expression into a postfix expression. It uses a stack; but in this case, the stack is used to hold operators rather than numbers. The purpose of the stack is to reverse the order of the operators in the expression. It also serves as a storage structure, since no operator can be printed until both of its operands have appeared.

## 2.1 Assumptions

- Able to handle both negative and positive integers
- The infix notation is parseable and in a consistent format
- There is a single space between all values, operators and parenthesis

## 2.1 Define the arithmetic function
The ask is to support a subset of all available mathematical operands. 

The order of operations (or operator precedence) is a collection of rules that reflect conventions about which procedures to perform first in order to evaluate a given mathematical expression.
The order is usually denoted using the acronymn <b>PEMDAS</b> which is as follows: parenthesis, exponents, multiplication, division, addition, subtraction. 

The relative precedence of each operation is not contiguous as in this example multiplication/division and addition/subtraction share the precedence values

In [1]:
ops = {
    '*': {'prec': 2,'op': lambda x,y: float(x) *  float(y)},
    '/': {'prec': 2,'op': lambda x,y: float(x) /  float(y)},
    '+': {'prec': 1,'op': lambda x,y: float(x) +  float(y)},
    '-': {'prec': 1,'op': lambda x,y: float(x) -  float(y)},
    '(': {'prec': 0},
}

### 2.3 Statement of arithmetic method
Due to the aforementioned order of operation we need to modify the mathematical notation.
In the problem statement we are given an expression written in the <b>infix</b> notation which has no notion of precedence. Moving to a new intermediate format called <b>postfix</b> is a standard way to address this.

Using this approach the expression <b>[1 + 2 * 3]</b> becomes <b>[1 2 3 + *]</b> and by applying additional logic involving parenthesis and precedence it becomes a trivial closed-form solution with linear. The approach then is essentially to parse the expression, extract the values and operations into two arrays and evaluate.

In [16]:
out_stack = []
ops_stack = []

LH = '(' 
RH = ')'

def algo(expression):
    print('INFIX:', expression)
    # While there are expression tokens to be read
    for token in expression.split(' '):
        # If it's a number add it to the output stack
        if token.lstrip("-").isdigit():
            out_stack.append(token)
        # If it's a left bracket
        elif token == LH:
            # Push it onto the operator stack
            ops_stack.append(token)
        # If it's a right bracket
        elif token == RH:
            # While there's not a left bracket at the top of the stack:
            while len(ops_stack) > 0 and ops_stack[-1] != LH:
                # Pop operators from the operator stack onto the output stack
                out_stack.append(ops_stack.pop())
            # Pop the left bracket from the operator stack and discard it
            ops_stack.pop()
        # If it's an operator
        elif token in ops:
            # While there's an operator on the top of the operator stack with greater precedence
            while len(ops_stack) > 0 and ops[ops_stack[-1]]['prec'] >= ops[token]['prec']:
                # Pop operators from the operator stack onto the output stack
                out_stack.append(ops_stack.pop())
            # Push the current operator onto the stack
            ops_stack.append(token)
    # While there are operators on the operator stack, pop them to the output stack       
    while ops_stack:
        out_stack.append(ops_stack.pop())
    return out_stack

def evaluate(expression, debug=False):
    if debug:
        print('RPN: %s' % ' '.join(expression))
    stack = []
    for token in expression:
        if token in ops:
            arg2 = stack.pop()
            arg1 = stack.pop()
            result = ops[token]['op'](arg1, arg2)
            stack.append(result)
        else:
            stack.append(int(token))
    return stack.pop()

expression = '1 + 3 / ( 4 * 2 ) / ( 1 - 5 )'
expression = '3 + ( ( 4 * 5 ) ) / ( 1 + 2 ) * 7 + 33 / ( 8 / 9 )'

evaluate(algo(expression), True)

INFIX: 3 + ( ( 4 * 5 ) ) / ( 1 + 2 ) * 7 + 33 / ( 8 / 9 )
RPN: 3 4 5 * 1 2 + / 7 * + 33 8 9 / / +


86.79166666666667